In [1]:
import pandas as pd
import nltk
from sklearn.utils import shuffle

# Loading our datafile as a DataFrame with pandas

df = pd.read_csv('/Users/laniepreston/TMCI_Edits/group_assignment/FullReviews.csv' , usecols=[1,6,9], header=0)
shuf_red_df = shuffle(df[:10000])
print(shuf_red_df)

       ProductId  Score                                               Text
7641  B000TRFGGM      5  We love the little Venus Flytrap,<br />it's he...
6776  B001LG945O      4  I'll admit that I don't care for drinks that a...
874   B000HDMUQ2      2  I have recently stopped eating a number of all...
9061  B006N3IG4K      5  I really enjoy Wolfgang Puck's K-Cups, so far ...
5003  B00653KJ5M      4  I bought a 40 sampler pack from the caveman ba...
4603  B003VXL14M      5  I love this tea!  Both the green and white var...
9162  B006N3IG4K      5  WOW! That's what my husband said when he taste...
9723  B001LXYA5Q      3  This tablet which is referred to as the miracl...
8563  B003VXFK44      5  I buy these with subscribe and save.  I like t...
2439  B0089SPDUW      5  So Far this is the only K-cup I've found thats...
5137  B0028VAQSU      5  What's not to like about Pamela's baking mix? ...
4775  B00139TT72      4  I've yet to find any of the dogs I know that l...
8607  B003VXFK44      5  

In [3]:
# PoS-tagging our dataset
train_df = shuf_red_df[:8000]
test_df = shuf_red_df[:2000]


train_tagged = []
test_tagged = []


for index, item in train_df.iterrows():
    product_id = item[0]
    score = item[1]
    review = item[2]
    review = review.split()
    rev = []
    for i in review:
        rev += nltk.pos_tag([i], tagset="universal")
    train_tagged.append([rev, score])

for index, item in test_df.iterrows():
    product_id = item[0]
    score = item[1]
    review = item[2]
    review = review.split()
    rev = []
    for i in review:
        rev += nltk.pos_tag([i], tagset="universal")
    test_tagged.append([rev, score])

In [4]:
print(test_tagged[:10])

[[[('I', 'PRON'), ('got', 'VERB'), ('this', 'DET'), ('on', 'ADP'), ('the', 'DET'), ('warehouse', 'NOUN'), ('deal', 'NOUN'), ('so', 'ADV'), ('I', 'PRON'), ("wasn't", 'NOUN'), ('expecting', 'VERB'), ('much.', 'NOUN'), ('However,', 'NOUN'), ('be', 'VERB'), ('advised', 'VERB'), ('you', 'PRON'), ('will', 'VERB'), ('get', 'VERB'), ('NO', 'DET'), ('regular', 'ADJ'), ('Donut', 'NOUN'), ('Shop', 'NOUN'), ('and', 'CONJ'), ('instead', 'ADV'), ('you', 'PRON'), ('will', 'VERB'), ('receive', 'NOUN'), ('over', 'ADP'), ('half', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('box', 'NOUN'), ('as', 'ADP'), ('decaf.', 'NOUN'), ('I', 'PRON'), ('love', 'NOUN'), ('the', 'DET'), ('flavors', 'NOUN'), ('of', 'ADP'), ('all', 'DET'), ('of', 'ADP'), ('these', 'DET'), ('though,', 'NOUN'), ('so', 'ADV'), ("that's", 'NOUN'), ('ok.', 'NOUN'), ('But', 'CONJ'), ('I', 'PRON'), ('will', 'VERB'), ('think', 'NOUN'), ('hard', 'ADJ'), ('about', 'ADP'), ('getting', 'VERB'), ('another', 'DET'), ('variety', 'NOUN'), ('pack.', 'NOUN')

In [18]:
shuf_cooccs_df = shuf_red_df[:1000]
#print(shuf_cooccs_df)
cooccs_text = []
for index, item in shuf_cooccs_df.iterrows():
    product_id = item[0]
    score = item[1]
    review = item[2]
    review = review.split()
    rev = []
    for i in review:
        rev += nltk.pos_tag([i], tagset="universal")
    cooccs_text.append([product_id, rev, score])

In [19]:
print(cooccs_text[2])

['B000HDMUQ2', [('I', 'PRON'), ('have', 'VERB'), ('recently', 'ADV'), ('stopped', 'VERB'), ('eating', 'VERB'), ('a', 'DET'), ('number', 'NOUN'), ('of', 'ADP'), ('allergens', 'NOUN'), ('(dairy,', 'NOUN'), ('soy,', 'NOUN'), ('nuts)', 'NOUN'), ('and', 'CONJ'), ('these', 'DET'), ('were', 'VERB'), ('one', 'NUM'), ('of', 'ADP'), ('the', 'DET'), ('few', 'ADJ'), ('snacks', 'NOUN'), ('which', 'DET'), ('fit', 'NOUN'), ('into', 'ADP'), ('my', 'PRON'), ('dietary', 'ADJ'), ('restrictions.', 'NOUN'), ('Unfortunately,', 'NOUN'), ('these', 'DET'), ('were', 'VERB'), ('pretty', 'ADV'), ('gross', 'ADJ'), ('-', '.'), ('miserable', 'ADJ'), ('texture', 'NOUN'), ('and', 'CONJ'), ('more', 'ADV'), ('cardboard', 'NOUN'), ('flavored', 'VERB'), ('than', 'ADP'), ('berry.', 'NOUN'), ('I', 'PRON'), ('threw', 'NOUN'), ('them', 'PRON'), ('away.', 'NOUN'), ('I', 'PRON'), ('would', 'VERB'), ('recommend', 'NOUN'), ('the', 'DET'), ('cocoa', 'NOUN'), ('ones', 'NOUN'), ('as', 'ADP'), ('they', 'PRON'), ('are', 'VERB'), ('muc

In [8]:
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
# Lemmatizing our dataset + filtering for stopwords (except negation)
training_dicts = dict((k,[]) for k in [1,2,3,4,5])
test_dicts = dict((k,[]) for k in [1,2,3,4,5])

stopwords = set(stopwords.words('english'))

un2wn_mapping = {"VERB" : wn.VERB, "NOUN" : wn.NOUN, "ADJ" : wn.ADJ, "ADV" : wn.ADV}

lemmatized = []

for review in train_tagged:
    rev = review[0]
    score = review[1]
    lem_rev = []
    for tup in rev:
        w, tag = tup
        
        if len(w) > 1 and w.isalpha():
            w = w.lower()
        else:
            continue
        
        if tag in [".", "X"]:
            continue
        elif w.lower() in stopwords:
            if w.lower() in ["not", "t", "no"]:  
                lemma = w.lower()
                tag = "NEGATION"
            else: 
                continue
        
        elif tag in un2wn_mapping.keys():
            lemma = nltk.WordNetLemmatizer().lemmatize(w, pos = un2wn_mapping[tag])
        else:
            lemma = nltk.WordNetLemmatizer().lemmatize(w)
        lem_rev.append("-".join([lemma, tag]))
    training_dicts[score].append(lem_rev)

In [9]:
test_dicts = dict((k,[]) for k in [1,2,3,4,5])
for review in test_tagged:
    rev = review[0]
    score = review[1]
    lem_rev = []
    for tup in rev:
        w, tag = tup
        
        if len(w) > 1 and w.isalpha():
            w = w.lower()
        else:
            continue
        if tag in [".", "X"]:
            continue
        elif w.lower() in stopwords:
            if w.lower() in ["not", "t", "no"]:  
                lemma = w.lower()
                tag = "NEGATION"
                lem_rev.append("-".join([lemma, tag]))
            else: 
                continue
        elif tag in un2wn_mapping.keys():
            lemma = nltk.WordNetLemmatizer().lemmatize(w, pos = un2wn_mapping[tag])
        else:
            lemma = nltk.WordNetLemmatizer().lemmatize(w)
        lem_rev.append("-".join([lemma, tag]))
    test_dicts[score].append(lem_rev)

In [20]:
cooccs_list = []
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
stopwords = set(stopwords.words('english'))

un2wn_mapping = {"VERB" : wn.VERB, "NOUN" : wn.NOUN, "ADJ" : wn.ADJ, "ADV" : wn.ADV}

for review in cooccs_text:
    product_id = review[0]
    rev = review[1]
    score = review[2]
    lem_rev = []
    
    for tup in rev:
        w, tag = tup
        
        if len(w) > 1 and w.isalpha():
            w = w.lower()
        else:
            continue
        if tag in [".", "X"]:
            continue
        elif w.lower() in stopwords:
            if w.lower() in ["not", "t", "no"]:  
                lemma = w.lower()
                tag = "NEGATION"
                lem_rev.append("-".join([lemma, tag]))
            else: 
                continue
        elif tag in un2wn_mapping.keys():
            lemma = nltk.WordNetLemmatizer().lemmatize(w, pos = un2wn_mapping[tag])
        else:
            lemma = nltk.WordNetLemmatizer().lemmatize(w)
        lem_rev.append("-".join([lemma, tag]))
        
    cooccs_list.append([product_id, score, lem_rev])

In [22]:
import pickle
# Save lemmatized text to a pickle file
#pickle.dump(training_dicts, open("training_dicts.txt", "wb"))
#pickle.dump(test_dicts, open("test_dicts.txt", "wb"))
pickle.dump(cooccs_list, open("cooccs_final_list.txt", "wb"))

In [21]:
print(cooccs_list[:10])

[['B000TRFGGM', 5, ['love-NOUN', 'little-ADJ', 'venus-NOUN', 'produce-VERB', 'new-ADJ', 'arrive-VERB', 'quickly-ADV', 'good-ADJ', 'positive-ADJ']], ['B001LG945O', 4, ['admit-NOUN', 'care-NOUN', 'drink-NOUN', 'carbonate-VERB', 'carbonation-NOUN', 'soda-NOUN', 'make-VERB', 'tend-NOUN', 'diet-NOUN', 'drink-NOUN', 'weird-NOUN', 'taste-NOUN', 'sure-NOUN', 'expect-VERB', 'drank-NOUN', 'carbonation-NOUN', 'mild-NOUN', 'like-ADP', 'light-NOUN', 'buzz-NOUN', 'mouth-NOUN', 'instead-ADV', 'lot-NOUN', 'carbonation-NOUN', 'hear-NOUN', 'content-NOUN', 'fizz-NOUN', 'drink-NOUN', 'orange-NOUN', 'tangerine-NOUN', 'flavor-NOUN', 'nice-ADJ', 'combo-NOUN', 'strong-ADJ', 'tangerine-NOUN', 'flavor-NOUN', 'though-ADP', 'someone-NOUN', 'like-ADP', 'sugar-NOUN', 'found-NOUN', 'tad-NOUN', 'sweet-NOUN', 'advertised-ADJ', 'no-NEGATION', 'no-NEGATION', 'sugar-NOUN', 'less-ADV', 'not-NEGATION', 'not-NEGATION', 'sure-NOUN', 'would-VERB', 'drink-NOUN', 'though-ADP', 'small-ADJ', 'imagine-NOUN', 'someone-NOUN', 'drink